# Chapter 14: Building a Complete DApp

---

Now that we've covered the fundamentals of smart contracts, development tools, and Web3 libraries, it's time to bring everything together. In this chapter, we'll build a **complete decentralized application (DApp)** from scratch—a simple marketplace where users can list items for sale and purchase them using cryptocurrency. We'll follow a structured approach: planning, smart contract development, frontend integration, testing, and deployment.

By the end of this chapter, you'll have a working DApp and a solid understanding of the entire development lifecycle.

---

## 14.1 Project Planning

### 14.1.1 Defining Requirements

Before writing any code, we need to define what our DApp will do. Our marketplace will have the following features:

- **List an item**: Any user can list an item for sale by providing a name, description, price (in ETH), and an image.
- **View listings**: Anyone can view all active listings.
- **Purchase an item**: A user can buy an item by paying the exact price. The payment is transferred to the seller, and the listing is marked as sold.
- **Cancel a listing**: The seller can cancel a listing if it hasn't been sold yet.
- **Events**: Emit events for listing, purchase, and cancellation.

**User stories:**
- As a seller, I want to list an item with details so that buyers can see it.
- As a buyer, I want to see all available items and their prices.
- As a buyer, I want to purchase an item securely, with payment going to the seller.
- As a seller, I want to cancel my listing if I change my mind.

**Non-functional requirements:**
- The smart contract must be secure (no reentrancy, proper access control).
- The frontend should be responsive and user-friendly.
- Images should be stored off-chain (IPFS) to minimize gas costs.
- The DApp should work with MetaMask.

### 14.1.2 Choosing the Tech Stack

Based on our requirements, we'll use:

| Component | Technology | Reason |
|-----------|------------|--------|
| **Smart Contract** | Solidity (v0.8.19) | Industry standard |
| **Development Framework** | Hardhat | Robust, extensible, great for testing |
| **Frontend Framework** | React (with Next.js) | Popular, enables server-side rendering for better SEO |
| **Web3 Library** | ethers.js | Clean API, excellent documentation |
| **Wallet Integration** | MetaMask (ethers.providers.Web3Provider) | Most widely used |
| **Decentralized Storage** | IPFS (via Pinata or NFT.Storage) | Store images off-chain |
| **Testing** | Mocha/Chai (Hardhat) + Solidity tests (Foundry optional) | Comprehensive testing |
| **Deployment** | Hardhat deploy scripts | Repeatable and configurable |

### 14.1.3 Architecture Design

```
┌─────────────────────────────────────────────────────────────┐
│                     USER'S BROWSER                          │
│  ┌───────────────────────────────────────────────────────┐  │
│  │                    React Frontend                      │  │
│  │  • List items form                                    │  │
│  │  • Display listings                                   │  │
│  │  • Purchase buttons                                   │  │
│  └───────────────────────┬───────────────────────────────┘  │
│                          │                                   │
│                          ▼                                   │
│  ┌───────────────────────────────────────────────────────┐  │
│  │                    ethers.js                           │  │
│  │  • Connect to MetaMask                                │  │
│  │  • Call contract functions                             │  │
│  │  • Listen to events                                    │  │
│  └───────────────────────┬───────────────────────────────┘  │
└──────────────────────────┼────────────────────────────────────┘
                           │
              ┌────────────┴────────────┐
              │                         │
              ▼                         ▼
┌─────────────────────────┐   ┌─────────────────────────┐
│     IPFS / Pinata       │   │    Ethereum Blockchain  │
│  • Store images         │   │  • Marketplace Contract │
│  • Return IPFS hash     │   │  • Listings, purchases  │
└─────────────────────────┘   └─────────────────────────┘
```

**Data flow:**
1. Seller uploads an image to IPFS → gets a hash (CID).
2. Seller submits listing transaction with metadata (name, description, price, IPFS hash).
3. Contract stores listing and emits event.
4. Frontend queries contract or listens to events to display listings.
5. Buyer sends transaction to purchase, including exact payment.
6. Contract transfers ETH to seller, marks listing as sold.

---

## 14.2 Smart Contract Development

### 14.2.1 Designing the Contract

We'll create a `Marketplace` contract with the following structure:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

contract Marketplace {
    struct Listing {
        uint id;
        address payable seller;
        string name;
        string description;
        uint price;          // price in wei
        string imageHash;    // IPFS hash
        bool active;
    }

    uint public nextListingId;
    mapping(uint => Listing) public listings;

    event ListingCreated(uint indexed id, address indexed seller, uint price, string name);
    event ListingPurchased(uint indexed id, address indexed buyer, uint price);
    event ListingCancelled(uint indexed id, address indexed seller);

    function listItem(string memory _name, string memory _description, uint _price, string memory _imageHash) external {
        require(_price > 0, "Price must be >0");
        require(bytes(_name).length > 0, "Name required");
        
        listings[nextListingId] = Listing({
            id: nextListingId,
            seller: payable(msg.sender),
            name: _name,
            description: _description,
            price: _price,
            imageHash: _imageHash,
            active: true
        });

        emit ListingCreated(nextListingId, msg.sender, _price, _name);
        nextListingId++;
    }

    function purchaseItem(uint _id) external payable {
        Listing storage listing = listings[_id];
        require(listing.active, "Item not for sale");
        require(msg.value >= listing.price, "Insufficient payment");
        require(listing.seller != msg.sender, "Cannot buy your own item");

        listing.active = false;
        listing.seller.transfer(listing.price);

        // Refund excess payment
        if (msg.value > listing.price) {
            payable(msg.sender).transfer(msg.value - listing.price);
        }

        emit ListingPurchased(_id, msg.sender, listing.price);
    }

    function cancelListing(uint _id) external {
        Listing storage listing = listings[_id];
        require(listing.seller == msg.sender, "Not your listing");
        require(listing.active, "Already sold or cancelled");

        listing.active = false;
        emit ListingCancelled(_id, msg.sender);
    }

    function getActiveListings() external view returns (Listing[] memory) {
        uint count = 0;
        for (uint i = 0; i < nextListingId; i++) {
            if (listings[i].active) {
                count++;
            }
        }

        Listing[] memory active = new Listing[](count);
        uint index = 0;
        for (uint i = 0; i < nextListingId; i++) {
            if (listings[i].active) {
                active[index] = listings[i];
                index++;
            }
        }
        return active;
    }
}
```

**Key design points:**
- `Listing` struct holds all item data.
- `nextListingId` auto-increments for unique IDs.
- `active` flag prevents double-selling.
- `purchaseItem` sends payment to seller and refunds excess.
- `cancelListing` only by seller.
- `getActiveListings` returns only currently available items (gas-inefficient for large arrays; in production you'd use an indexer like The Graph).

### 14.2.2 Writing and Testing the Contract

We'll use Hardhat for development and testing.

**Initialize project:**
```bash
mkdir marketplace-dapp
cd marketplace-dapp
npm init -y
npm install --save-dev hardhat @nomicfoundation/hardhat-toolbox @openzeppelin/contracts
npx hardhat init
```

**Place the contract in `contracts/Marketplace.sol`.**

**Write tests (`test/Marketplace.js`):**

```javascript
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("Marketplace", function () {
  let marketplace;
  let seller, buyer, other;

  beforeEach(async function () {
    [seller, buyer, other] = await ethers.getSigners();
    const Marketplace = await ethers.getContractFactory("Marketplace");
    marketplace = await Marketplace.deploy();
    await marketplace.waitForDeployment();
  });

  describe("Listing", function () {
    it("Should create a listing", async function () {
      const price = ethers.parseEther("1");
      const tx = await marketplace.connect(seller).listItem(
        "Item 1",
        "Description",
        price,
        "QmHash"
      );
      await tx.wait();

      const listing = await marketplace.listings(0);
      expect(listing.name).to.equal("Item 1");
      expect(listing.seller).to.equal(seller.address);
      expect(listing.price).to.equal(price);
      expect(listing.active).to.be.true;
    });

    it("Should not allow zero price", async function () {
      await expect(
        marketplace.connect(seller).listItem("Item", "Desc", 0, "hash")
      ).to.be.revertedWith("Price must be >0");
    });
  });

  describe("Purchase", function () {
    beforeEach(async function () {
      const price = ethers.parseEther("1");
      await marketplace.connect(seller).listItem("Item", "Desc", price, "hash");
    });

    it("Should allow purchase with exact price", async function () {
      const sellerBalanceBefore = await ethers.provider.getBalance(seller.address);
      const price = ethers.parseEther("1");

      const tx = await marketplace.connect(buyer).purchaseItem(0, { value: price });
      await tx.wait();

      const listing = await marketplace.listings(0);
      expect(listing.active).to.be.false;

      const sellerBalanceAfter = await ethers.provider.getBalance(seller.address);
      expect(sellerBalanceAfter - sellerBalanceBefore).to.equal(price);
    });

    it("Should refund excess payment", async function () {
      const price = ethers.parseEther("1");
      const overpay = ethers.parseEther("1.5");

      const buyerBalanceBefore = await ethers.provider.getBalance(buyer.address);

      const tx = await marketplace.connect(buyer).purchaseItem(0, { value: overpay });
      await tx.wait();

      const buyerBalanceAfter = await ethers.provider.getBalance(buyer.address);
      // Buyer paid 1.5, got 0.5 refund, net loss = 1 ETH + gas
      expect(buyerBalanceAfter).to.be.lessThan(buyerBalanceBefore - price);
    });

    it("Should not allow buying own item", async function () {
      const price = ethers.parseEther("1");
      await expect(
        marketplace.connect(seller).purchaseItem(0, { value: price })
      ).to.be.revertedWith("Cannot buy your own item");
    });
  });

  describe("Cancel", function () {
    it("Should allow seller to cancel", async function () {
      await marketplace.connect(seller).listItem("Item", "Desc", 1, "hash");
      await marketplace.connect(seller).cancelListing(0);

      const listing = await marketplace.listings(0);
      expect(listing.active).to.be.false;
    });

    it("Should not allow non-seller to cancel", async function () {
      await marketplace.connect(seller).listItem("Item", "Desc", 1, "hash");
      await expect(
        marketplace.connect(buyer).cancelListing(0)
      ).to.be.revertedWith("Not your listing");
    });
  });
});
```

Run tests:
```bash
npx hardhat test
```

**Output should show all tests passing.**

### 14.2.3 Security Auditing

Before deploying, consider the following security aspects:

- **Reentrancy**: In `purchaseItem`, we send ETH (`seller.transfer`) after updating state (`listing.active = false`). This follows checks-effects-interactions pattern, so safe.
- **Access control**: `cancelListing` checks `listing.seller == msg.sender`.
- **Integer overflow**: Solidity 0.8+ includes built-in overflow checks.
- **Denial of service**: None apparent.
- **Front-running**: Not a concern for this simple marketplace.

For production, you'd also consider:
- Using OpenZeppelin's `ReentrancyGuard` for extra safety.
- Adding a circuit breaker (pause) in case of emergency.
- Considering gas efficiency (the `getActiveListings` loop could become expensive; better to use an indexer).

We'll assume the contract is secure for our learning purposes.

---

## 14.3 Frontend Development

Now we'll build the frontend using React (with Next.js) to interact with our contract.

### 14.3.1 Setting Up React/Next.js

**Create a Next.js app:**
```bash
npx create-next-app@latest marketplace-frontend
cd marketplace-frontend
npm install ethers @web3-react/core @web3-react/injected-connector
```

**Clean up default files** and create the following structure:

```
pages/
  index.js          # Main marketplace page
  _app.js           # App wrapper with Web3 provider
components/
  Header.js         # Wallet connection button
  ListingForm.js    # Form to create new listing
  ListingCard.js    # Display a single listing
  ListingsGrid.js   # Grid of all listings
context/
  Web3Context.js    # Web3 provider context
```

### 14.3.2 Wallet Integration (MetaMask)

We'll use `@web3-react` for managing wallet connection, but for simplicity, we'll create a custom React context.

**Create `context/Web3Context.js`:**
```javascript
import { createContext, useContext, useEffect, useState } from 'react';
import { ethers } from 'ethers';

const Web3Context = createContext();

export function Web3Provider({ children }) {
  const [provider, setProvider] = useState(null);
  const [signer, setSigner] = useState(null);
  const [account, setAccount] = useState(null);
  const [chainId, setChainId] = useState(null);
  const [error, setError] = useState(null);

  const connectWallet = async () => {
    if (typeof window.ethereum === 'undefined') {
      setError('Please install MetaMask');
      return;
    }

    try {
      const web3Provider = new ethers.providers.Web3Provider(window.ethereum);
      await window.ethereum.request({ method: 'eth_requestAccounts' });
      const signer = web3Provider.getSigner();
      const account = await signer.getAddress();
      const network = await web3Provider.getNetwork();

      setProvider(web3Provider);
      setSigner(signer);
      setAccount(account);
      setChainId(network.chainId);
    } catch (err) {
      setError(err.message);
    }
  };

  useEffect(() => {
    if (window.ethereum) {
      window.ethereum.on('accountsChanged', (accounts) => {
        if (accounts.length > 0) {
          setAccount(accounts[0]);
          // Reconnect or update signer
        } else {
          setAccount(null);
          setSigner(null);
        }
      });

      window.ethereum.on('chainChanged', () => {
        window.location.reload();
      });
    }
  }, []);

  return (
    <Web3Context.Provider value={{
      provider, signer, account, chainId, error,
      connectWallet
    }}>
      {children}
    </Web3Context.Provider>
  );
}

export const useWeb3 = () => useContext(Web3Context);
```

**Wrap `_app.js` with the provider:**
```javascript
import { Web3Provider } from '../context/Web3Context';

function MyApp({ Component, pageProps }) {
  return (
    <Web3Provider>
      <Component {...pageProps} />
    </Web3Provider>
  );
}

export default MyApp;
```

### 14.3.3 Contract Integration with ethers.js

We need the contract address and ABI. After deploying (later), we'll have these. For now, we'll store them in a config file.

**Create `config.js`:**
```javascript
export const MARKETPLACE_ADDRESS = '0x...'; // To be filled after deployment
export const MARKETPLACE_ABI = [ /* ABI from compilation */ ];
```

We'll create a custom hook to interact with the contract.

**Create `hooks/useMarketplace.js`:**
```javascript
import { useState, useEffect } from 'react';
import { ethers } from 'ethers';
import { useWeb3 } from '../context/Web3Context';
import { MARKETPLACE_ADDRESS, MARKETPLACE_ABI } from '../config';

export function useMarketplace() {
  const { provider, signer, account } = useWeb3();
  const [contract, setContract] = useState(null);
  const [listings, setListings] = useState([]);
  const [loading, setLoading] = useState(false);

  useEffect(() => {
    if (signer) {
      const marketplace = new ethers.Contract(
        MARKETPLACE_ADDRESS,
        MARKETPLACE_ABI,
        signer
      );
      setContract(marketplace);
    } else if (provider) {
      // Read-only instance
      const marketplace = new ethers.Contract(
        MARKETPLACE_ADDRESS,
        MARKETPLACE_ABI,
        provider
      );
      setContract(marketplace);
    }
  }, [signer, provider]);

  const loadListings = async () => {
    if (!contract) return;
    setLoading(true);
    try {
      const activeListings = await contract.getActiveListings();
      setListings(activeListings.map(l => ({
        id: l.id.toString(),
        seller: l.seller,
        name: l.name,
        description: l.description,
        price: ethers.utils.formatEther(l.price),
        imageHash: l.imageHash,
        active: l.active
      })));
    } catch (err) {
      console.error('Failed to load listings', err);
    } finally {
      setLoading(false);
    }
  };

  const listItem = async (name, description, priceEth, imageHash) => {
    if (!contract || !signer) throw new Error('Not connected');
    const priceWei = ethers.utils.parseEther(priceEth);
    const tx = await contract.listItem(name, description, priceWei, imageHash);
    await tx.wait();
    await loadListings(); // refresh
  };

  const purchaseItem = async (id, priceEth) => {
    if (!contract || !signer) throw new Error('Not connected');
    const priceWei = ethers.utils.parseEther(priceEth);
    const tx = await contract.purchaseItem(id, { value: priceWei });
    await tx.wait();
    await loadListings();
  };

  const cancelItem = async (id) => {
    if (!contract || !signer) throw new Error('Not connected');
    const tx = await contract.cancelListing(id);
    await tx.wait();
    await loadListings();
  };

  // Load listings on contract change
  useEffect(() => {
    if (contract) {
      loadListings();
      // Optionally listen to events
      contract.on('ListingCreated', loadListings);
      contract.on('ListingPurchased', loadListings);
      contract.on('ListingCancelled', loadListings);
      return () => {
        contract.removeAllListeners();
      };
    }
  }, [contract]);

  return { listings, loading, listItem, purchaseItem, cancelItem, loadListings };
}
```

### 14.3.4 State Management

We're using React hooks for local state; the `useMarketplace` hook manages listings and contract interactions. For more complex apps, you might use Redux or Zustand, but for this example, context and hooks are sufficient.

---

## 14.4 Building a Decentralized Marketplace

Now let's assemble the components into a functional marketplace.

### 14.4.1 Smart Contract Implementation

We already wrote the contract in section 14.2.1. We'll deploy it to a testnet later.

### 14.4.2 Frontend Implementation

**Header component (`components/Header.js`):**
```jsx
import { useWeb3 } from '../context/Web3Context';

export default function Header() {
  const { account, connectWallet, error } = useWeb3();

  return (
    <header style={styles.header}>
      <h1>Decentralized Marketplace</h1>
      <div>
        {account ? (
          <span>Connected: {account.slice(0,6)}...{account.slice(-4)}</span>
        ) : (
          <button onClick={connectWallet} style={styles.button}>
            Connect Wallet
          </button>
        )}
        {error && <p style={styles.error}>{error}</p>}
      </div>
    </header>
  );
}

const styles = {
  header: { display: 'flex', justifyContent: 'space-between', padding: '1rem', background: '#f0f0f0' },
  button: { padding: '0.5rem 1rem' },
  error: { color: 'red' }
};
```

**ListingForm component (`components/ListingForm.js`):**
```jsx
import { useState } from 'react';
import { useMarketplace } from '../hooks/useMarketplace';
import { uploadToIPFS } from '../utils/ipfs'; // we'll implement this

export default function ListingForm() {
  const { listItem } = useMarketplace();
  const [name, setName] = useState('');
  const [description, setDescription] = useState('');
  const [price, setPrice] = useState('');
  const [image, setImage] = useState(null);
  const [uploading, setUploading] = useState(false);
  const [submitting, setSubmitting] = useState(false);

  const handleImageUpload = async (e) => {
    const file = e.target.files[0];
    if (!file) return;
    setUploading(true);
    try {
      const hash = await uploadToIPFS(file);
      setImage(hash);
    } catch (err) {
      alert('Upload failed');
    } finally {
      setUploading(false);
    }
  };

  const handleSubmit = async (e) => {
    e.preventDefault();
    if (!name || !description || !price || !image) {
      alert('Please fill all fields and upload an image');
      return;
    }
    setSubmitting(true);
    try {
      await listItem(name, description, price, image);
      alert('Listing created!');
      setName(''); setDescription(''); setPrice(''); setImage(null);
    } catch (err) {
      alert(err.message);
    } finally {
      setSubmitting(false);
    }
  };

  return (
    <form onSubmit={handleSubmit} style={styles.form}>
      <h2>List an Item</h2>
      <input
        type="text"
        placeholder="Item Name"
        value={name}
        onChange={(e) => setName(e.target.value)}
        required
      />
      <textarea
        placeholder="Description"
        value={description}
        onChange={(e) => setDescription(e.target.value)}
        required
      />
      <input
        type="number"
        step="0.001"
        placeholder="Price (ETH)"
        value={price}
        onChange={(e) => setPrice(e.target.value)}
        required
      />
      <div>
        <label>Image:</label>
        <input type="file" accept="image/*" onChange={handleImageUpload} />
        {uploading && <p>Uploading...</p>}
        {image && <p>Uploaded: {image}</p>}
      </div>
      <button type="submit" disabled={uploading || submitting}>
        {submitting ? 'Creating...' : 'Create Listing'}
      </button>
    </form>
  );
}

const styles = {
  form: { display: 'flex', flexDirection: 'column', gap: '1rem', maxWidth: '500px', margin: '2rem auto' }
};
```

**IPFS upload utility (`utils/ipfs.js`):**
```javascript
// Using Pinata API for simplicity
import axios from 'axios';

export async function uploadToIPFS(file) {
  const url = 'https://api.pinata.cloud/pinning/pinFileToIPFS';
  const formData = new FormData();
  formData.append('file', file);

  const res = await axios.post(url, formData, {
    maxBodyLength: 'Infinity',
    headers: {
      'Content-Type': `multipart/form-data`,
      pinata_api_key: process.env.NEXT_PUBLIC_PINATA_API_KEY,
      pinata_secret_api_key: process.env.NEXT_PUBLIC_PINATA_SECRET_KEY
    }
  });

  return res.data.IpfsHash;
}
```

**ListingCard component (`components/ListingCard.js`):**
```jsx
import { useMarketplace } from '../hooks/useMarketplace';
import { useWeb3 } from '../context/Web3Context';

export default function ListingCard({ listing }) {
  const { purchaseItem, cancelItem } = useMarketplace();
  const { account } = useWeb3();

  const handleBuy = async () => {
    if (!confirm(`Buy ${listing.name} for ${listing.price} ETH?`)) return;
    try {
      await purchaseItem(listing.id, listing.price);
    } catch (err) {
      alert(err.message);
    }
  };

  const handleCancel = async () => {
    if (!confirm('Cancel this listing?')) return;
    try {
      await cancelItem(listing.id);
    } catch (err) {
      alert(err.message);
    }
  };

  return (
    <div style={styles.card}>
      <img
        src={`https://gateway.pinata.cloud/ipfs/${listing.imageHash}`}
        alt={listing.name}
        style={styles.image}
      />
      <h3>{listing.name}</h3>
      <p>{listing.description}</p>
      <p><strong>{listing.price} ETH</strong></p>
      <p>Seller: {listing.seller.slice(0,6)}...{listing.seller.slice(-4)}</p>
      {account?.toLowerCase() === listing.seller.toLowerCase() ? (
        <button onClick={handleCancel} style={styles.cancel}>Cancel</button>
      ) : (
        <button onClick={handleBuy} style={styles.buy}>Buy</button>
      )}
    </div>
  );
}

const styles = {
  card: { border: '1px solid #ccc', padding: '1rem', borderRadius: '8px', width: '250px' },
  image: { width: '100%', height: '200px', objectFit: 'cover', borderRadius: '4px' },
  buy: { backgroundColor: '#4CAF50', color: 'white', border: 'none', padding: '0.5rem 1rem', cursor: 'pointer' },
  cancel: { backgroundColor: '#f44336', color: 'white', border: 'none', padding: '0.5rem 1rem', cursor: 'pointer' }
};
```

**ListingsGrid component (`components/ListingsGrid.js`):**
```jsx
import ListingCard from './ListingCard';

export default function ListingsGrid({ listings }) {
  if (!listings || listings.length === 0) {
    return <p>No active listings</p>;
  }

  return (
    <div style={styles.grid}>
      {listings.map(listing => (
        <ListingCard key={listing.id} listing={listing} />
      ))}
    </div>
  );
}

const styles = {
  grid: { display: 'grid', gridTemplateColumns: 'repeat(auto-fill, minmax(250px, 1fr))', gap: '1rem', padding: '1rem' }
};
```

**Main page (`pages/index.js`):**
```jsx
import { useMarketplace } from '../hooks/useMarketplace';
import Header from '../components/Header';
import ListingForm from '../components/ListingForm';
import ListingsGrid from '../components/ListingsGrid';

export default function Home() {
  const { listings, loading } = useMarketplace();

  return (
    <div>
      <Header />
      <ListingForm />
      <h2 style={{ paddingLeft: '1rem' }}>Active Listings</h2>
      {loading ? <p>Loading...</p> : <ListingsGrid listings={listings} />}
    </div>
  );
}
```

### 14.4.3 IPFS Integration for Images

We used Pinata to upload images. In production, you might want to allow users to upload directly to IPFS via browser using `ipfs-http-client` or a service like NFT.Storage. We've used environment variables for API keys (should be set in `.env.local`).

### 14.4.4 Complete Walkthrough

With all pieces in place, here's how the user experience flows:

1. User connects wallet via Header button.
2. User fills out the listing form, uploads an image (to IPFS).
3. On submit, a transaction is sent to the marketplace contract, creating a listing.
4. The frontend listens for the `ListingCreated` event and refreshes the listings.
5. The new listing appears in the grid.
6. Another user clicks "Buy", confirms, and sends a transaction with the exact ETH.
7. The contract transfers ETH to seller; frontend updates.
8. Seller can cancel their own listing.

---

## 14.5 Testing and Deployment

### 14.5.1 Unit Testing Smart Contracts

We already wrote unit tests in Hardhat. We should also test edge cases and potential exploits. We can add tests for:

- Canceling a sold item should fail.
- Purchasing with insufficient funds should revert.
- Multiple listings, pagination (though we don't have pagination).
- Event emission.

### 14.5.2 Integration Testing

Integration testing involves testing the entire flow with a local blockchain. We can use Hardhat's mainnet forking to simulate real-world conditions, or simply run tests on a local Hardhat node.

We can also write end-to-end tests using Playwright or Cypress that interact with a locally deployed contract and a local frontend.

### 14.5.3 Deploying to Testnets (Sepolia, Goerli)

We'll deploy to Sepolia testnet. First, get some test ETH from a faucet.

**Create a `.env` file in the Hardhat project root:**
```
PRIVATE_KEY=your_wallet_private_key
SEPOLIA_RPC_URL=https://sepolia.infura.io/v3/your_project_id
ETHERSCAN_API_KEY=your_etherscan_api_key
```

**Update `hardhat.config.js`:**
```javascript
require('@nomicfoundation/hardhat-toolbox');
require('dotenv').config();

module.exports = {
  solidity: '0.8.19',
  networks: {
    sepolia: {
      url: process.env.SEPOLIA_RPC_URL,
      accounts: [process.env.PRIVATE_KEY]
    }
  },
  etherscan: {
    apiKey: process.env.ETHERSCAN_API_KEY
  }
};
```

**Create a deployment script (`scripts/deploy.js`):**
```javascript
const { ethers } = require("hardhat");

async function main() {
  const Marketplace = await ethers.getContractFactory("Marketplace");
  const marketplace = await Marketplace.deploy();
  await marketplace.waitForDeployment();

  console.log("Marketplace deployed to:", await marketplace.getAddress());
}

main().catch(console.error);
```

**Deploy:**
```bash
npx hardhat run scripts/deploy.js --network sepolia
```

Copy the deployed address into the frontend's `config.js`.

### 14.5.4 Deploying to Mainnet

Deploying to mainnet is the same as testnet, but you use mainnet RPC and ensure your contract is thoroughly audited. Always test on testnet first.

### 14.5.5 Verification on Etherscan

After deployment, verify the contract on Etherscan (or Sepolia Etherscan) for transparency.

```bash
npx hardhat verify --network sepolia DEPLOYED_ADDRESS
```

This makes your source code public and allows users to interact with the contract via Etherscan's interface.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 14 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  In this chapter, we built a complete decentralized marketplace:│
│                                                                 │
│  • Defined requirements and chose tech stack                    │
│  • Designed and implemented a Solidity smart contract           │
│  • Wrote comprehensive tests with Hardhat                       │
│  • Built a React frontend with wallet connection and IPFS       │
│  • Integrated ethers.js for contract interaction                │
│  • Deployed to Sepolia testnet and verified on Etherscan        │
│                                                                 │
│  KEY TAKEAWAYS:                                                 │
│  • Always follow checks-effects-interactions pattern            │
│  • Store large files off-chain (IPFS)                           │
│  • Use events to keep frontend in sync                          │
│  • Test thoroughly, including edge cases                        │
│  • Deploy to testnet first, then mainnet                        │
│                                                                 │
│  EXTENSIONS:                                                    │
│  • Add pagination to avoid gas issues                           │
│  • Use The Graph for efficient querying                         │
│  • Implement royalty fees for creators                          │
│  • Add an auction mechanism                                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 15 – Wallet Integration and User Authentication. We'll dive deeper into wallet connection, signing messages, and implementing Sign-In with Ethereum (SIWE) for passwordless authentication.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='13. web3js_and_ethersjs_libraries.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='15. wallet_integration_and_user_authentication.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
